In [2]:
% pylab inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
from scipy.io.wavfile import read as sciread
from scipy import stats

Populating the interactive namespace from numpy and matplotlib


In [3]:
def calcFFT(audio, fs):
    transformed = np.fft.fft(audio)
    f = np.arange(0, len(transformed))*fs/len(transformed)
    f = f[:44100]
    transformed = transformed[:44100]
    return (f, transformed)

In [4]:
def plotFFT(audio, fs):
    f, transformed = calcFFT(audio, fs)
    plt.plot(f, np.abs(transformed))
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Magnitude (dB)')
    plt.show()

In [38]:
def get_tempo(signal, sr):
    onset_env = librosa.onset.onset_strength(signal, sr=sr)
    tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)
    return tempo

In [5]:
def get_harmonic(signal, sr):
    signal = librosa.effects.harmonic(signal)
    f, transformed = calcFFT(signal, sr)
    transformed = np.array(np.abs(transformed))
    data = {}
    data['hmedian'] = np.median(transformed)
    data['hmean'] = np.mean(transformed)
    data['h1Q'] = np.percentile(transformed, 25)
    data['h2Q'] = np.percentile(transformed, 75)
    data['hIQR'] = stats.iqr(transformed)
    data['hmin'] = np.min(transformed)
    data['hmax'] = np.max(transformed)
    data['hstd'] = np.std(transformed)
    data['htempo'] = get_tempo(signal, 44100)
    return data

In [6]:
def get_percussion(signal, sr):
    signal = librosa.effects.percussive(signal)
    f, transformed = calcFFT(signal, sr)
    transformed = np.array(np.abs(transformed))
    data = {}
    data['pmedian'] = np.median(transformed)
    data['pmean'] = np.mean(transformed)
    data['p1Q'] = np.percentile(transformed, 25)
    data['p2Q'] = np.percentile(transformed, 75)
    data['pIQR'] = stats.iqr(transformed)
    data['pmin'] = np.min(transformed)
    data['pmax'] = np.max(transformed)
    data['pstd'] = np.std(transformed)
    data['ptempo'] = get_tempo(signal, 44100)
    return data

In [7]:
def get_info(fpath, label):
    signal = librosa.core.load(fpath, 44100)[0]
    f, transformed = calcFFT(signal, 44100)
    transformed = np.array(np.abs(transformed))
    harmonic_data = get_harmonic(signal, 44100)
    percussion_data = get_percussion(signal, 44100)
    data = {}
    data['fpath'] = fpath
    data['label'] = label
    data['median'] = np.median(transformed)
    data['mean'] = np.mean(transformed)
    data['1Q'] = np.percentile(transformed, 25)
    data['2Q'] = np.percentile(transformed, 75)
    data['IQR'] = stats.iqr(transformed)
    data['min'] = np.min(transformed)
    data['max'] = np.max(transformed)
    data['std'] = np.std(transformed)
    data['tempo'] = get_tempo(signal, 44100)
    data = {**data, **percussion_data, **harmonic_data}
    return data

In [23]:
test = pd.read_csv('usefull_test.csv')
test.head()

,Unnamed: 0,Unnamed: 0.1,fname,label,fpath
0,0,4,00326aa9.wav,Oboe,../audio_test/00326aa9.wav
1,1,5,0038a046.wav,Bass_drum,../audio_test/0038a046.wav
2,2,8,007759c4.wav,Saxophone,../audio_test/007759c4.wav
3,3,9,008afd93.wav,Saxophone,../audio_test/008afd93.wav
4,4,12,00ae03f6.wav,Chime,../audio_test/00ae03f6.wav


In [31]:
working = pd.DataFrame()

In [32]:
import time

In [33]:
time_list = []

In [39]:
initial_time = time.time()
for i in test.index:
    fpath = test.loc[i, 'fpath']
    label = test.loc[i, 'label']
    info = get_info(fpath, label)
    df = pd.DataFrame(info, index=[i])
    working = pd.concat([working, df])
    if i % 50 == 0:
        new_time = time.time()
        time_difference = round(new_time - initial_time, 2)
        time_list.append(time_difference)
        print('Iteration: {}\n Local Time: {}. Time for the last 50 iterations: {} seconds'.format(i, time.asctime(time.localtime(new_time)), time_difference))
        print('Average time to run 50 iterations: {} seconds'.format(round(np.mean(time_list), 2)))
        initial_time = new_time

Iteration: 0
 Local Time: Wed Nov  7 01:03:26 2018. Time for the last 50 iterations: 1.45 seconds
Average time to run 50 iterations: 1.45 seconds
Iteration: 50
 Local Time: Wed Nov  7 01:05:41 2018. Time for the last 50 iterations: 135.02 seconds
Average time to run 50 iterations: 68.24 seconds
Iteration: 100
 Local Time: Wed Nov  7 01:07:46 2018. Time for the last 50 iterations: 124.76 seconds
Average time to run 50 iterations: 87.08 seconds
Iteration: 150
 Local Time: Wed Nov  7 01:10:26 2018. Time for the last 50 iterations: 159.61 seconds
Average time to run 50 iterations: 105.21 seconds
Iteration: 200
 Local Time: Wed Nov  7 01:13:18 2018. Time for the last 50 iterations: 172.11 seconds
Average time to run 50 iterations: 118.59 seconds
Iteration: 250
 Local Time: Wed Nov  7 01:15:51 2018. Time for the last 50 iterations: 153.46 seconds
Average time to run 50 iterations: 124.4 seconds
Iteration: 300
 Local Time: Wed Nov  7 01:18:06 2018. Time for the last 50 iterations: 134.66 seco

In [41]:
working.to_csv(path_or_buf='test_info.csv')